<a href="https://colab.research.google.com/github/MatiTaila/hack_iia/blob/ia-2003/face_detector/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Montar disco de Google Drive

Para proder acceder a datos guardados en su google drive desde un notebook de colab es necesario ejecutar el siguiente código

Esto va a ser necesario en todos los notebooks de este curso.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

# Testeo del algoritmo de reconocimiento facial

Finalmente llegamos a la parte en que evaluamos el sistema de detección y reconocimiento facial.

Vamos a utilizar el modelo que ya entrenamos para validar sobre nuestros datos de test. 

Para mostrar el reconocimiento de forma más amigable, lo primero que debemos hacer es asignarle un nombre a cada identificador. Para ello crearemos un vector de nombres, donde en cada posición correspondiente a cada identificador pondremos el nombre de la persona. Por ejemplo, como la persona asociada al identificador 1 es _Jennifer Aniston_ y la persona asociada al identificador 2 es _Bradd Pitt_ , armaremos nuestro vector como:
```
names = ['', 'Jennifer Aniston','Brad Pitt']

```
Notar que debemos dejar el primer elemento vacío, ya que los índices en `Python` comienzan en 0, y nosotros pusimos el primer identificador en 1.

In [0]:
import cv2
import numpy as np
import os
from glob import glob
import matplotlib.pyplot as plt

# Definimos Carpetas y Archivos a Utilizar
TEST_SET_PATH = '/content/drive/My Drive/hack_iia/face_detector/dataset/test'
CASCADE_FILE = '/content/drive/My Drive/hack_iia/face_detector/default.xml'
TRAINER_FILE = '/content/drive/My Drive/hack_iia/face_detector/trainer.yml'

# Lista de nombres
names = ['','Jennifer Aniston','Brad Pitt']

# Inicializamos el detector de caras
faceCascade = cv2.CascadeClassifier(CASCADE_FILE)

# Inicializamos el reconocedor de caras
reconocedor = cv2.face.LBPHFaceRecognizer_create()
reconocedor.read(TRAINER_FILE)

font = cv2.FONT_HERSHEY_SIMPLEX

# Creamos una grilla de gráficas para poder mostrar la salida
cols=5
rows=7
fig, ax = plt.subplots(rows,cols, figsize=(30,50))

labels = []
predictions = []

for img_count, image_path in enumerate(glob(TEST_SET_PATH + '/*.jpg')):
    
    img_name = os.path.basename(image_path)
    img = cv2.imread(image_path)
    minW = 0.2 * img.shape[0]
    minH = 0.2 * img.shape[1]
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Detectamos caras
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.3,
        minNeighbors=5,
        minSize=(int(minW), int(minH)))
    
    # Para cada cara detectada
    for (x, y, w, h) in faces:

        # Guardamos cual es la verdadera identidad de cada cara
        if img_name[0]=='b':
            labels.append(2)
        elif img_name[0]=='j':
            labels.append(1)

        # Ponemos un rectángulo alrededor
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 3)
        # Ejecutamos el reconocimiento facial
        id, dist = reconocedor.predict(gray[y:y + h, x:x + w])
        # Checkeamos si la probabilidad es muy baja, decimos que no podemos reconocer ninguna cara
        if (dist < 80):
            #Guardamos el valor de la prediccion
            predictions.append(id)
            id = names[id]
        else:
            id = names[0]
        # Superponemos el nombre y el score del clasificador
        cv2.putText(img, str(id), (x + 5, y - 5), font, 1, (255, 255, 255), 3)
        cv2.putText(img, '%.3f' % dist, (5, 30), font, 1, (0, 255, 255), 3)


    ax[img_count//cols][img_count%cols].imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
    ax[img_count//cols][img_count%cols].set_xticks([])
    ax[img_count//cols][img_count%cols].set_yticks([])

plt.show()


## Evaluación Simple ##

### Detección de Caras ###


In [0]:
caras_totales = 35
caras_detectadas = --
caras_detectadas_true_positive = --
caras_detectadas_false_positive = --
caras_detectadas_false_negative = --

In [0]:
PRECISION = caras_detectadas_true_positive/(caras_detectadas_true_positive + caras_detectadas_false_positive)

In [0]:
RECALL = caras_detectadas_true_positive/(caras_detectadas_true_positive + caras_detectadas_false_negative)

In [0]:
print('PRECISON: {:f}'.format(PRECISION))

In [0]:
print('RECALL: {:f}'.format(RECALL))

### Reconocimeinto Facial ###

In [0]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

confusion_mtx = confusion_matrix(labels, predictions) 
sns.heatmap(confusion_mtx, annot=True, fmt="d", xticklabels=names[1:],yticklabels=names[:1], cbar=False);

In [0]:
recognition_accuracy = sum(np.asarray(labels) == np.asarray(predictions))/len(predictions)*100

In [0]:
print("Recognition Accuracy: {}%".format(recognition_accuracy))

## FIN